In [ ]:
!pip install pycaret

In [ ]:
import pandas as pd
import pycaret.classification as pycr
import pycaret.utils as pycu
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
from plotly.subplots import make_subplots
import os
from pycaret.classification import *


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Use Augmentation File for training

In [ ]:
df = pd.read_csv(r"/content/drive/MyDrive/Breast_Segmentation_features/augmentation_approach/augmentation work/cc and mlo segments/sme+mixup.csv")
# df.sort_values(by='Date_Time')
#df = df.drop(df.loc[:, ['volume', 'case', 'image_name', 'side','view','segment']], axis=1)
display(df)


In [ ]:
df_train = df

In [ ]:
df_train.shape

In [ ]:
df_val = pd.read_csv(r"/content/drive/MyDrive/Breast_Segmentation_features/augmentation_approach/augmentation work/cc and mlo segments/cc&mlo_segments_test_10.csv")
# df.sort_values(by='Date_Time')
#df_val = df_val.drop(df_val.loc[:, ['volume', 'case', 'image_name', 'side','view','segment']], axis=1)
df_val = df_val.drop(df_val.loc[:, ['volume', 'case', 'image_name', 'side','view','segment']], axis=1)
display(df_val)

In [ ]:
### Select PCA or no PCA
model_preprocess = pycr.setup(data = df_train, test_data=df_val, target = 'diagnosis', numeric_features=['Angular Second Moment_0', 'Angular Second Moment_1', 'Angular Second Moment_2', 'Angular Second Moment_3',	'Contrast_0', 'Contrast_1', 'Contrast_2', 'Contrast_3','Correlation_0','Correlation_1', 'Correlation_2', 'Correlation_3','Sum of Squares Variance_0','Sum of Squares Variance_1', 'Sum of Squares Variance_2', 'Sum of Squares Variance_3','Inverse Difference Moment_0','Inverse Difference Moment_1','Inverse Difference Moment_2', 'Inverse Difference Moment_3','Sum Average_0', 'Sum Average_1','Sum Average_2', 'Sum Average_3','Sum Variance_0',	'Sum Variance_1', 'Sum Variance_2', 'Sum Variance_3','Sum Entropy_0','Sum Entropy_1', 'Sum Entropy_2', 'Sum Entropy_3',	'Entropy_0','Entropy_1','Entropy_2', 'Entropy_3','Difference Variance_0','Difference Variance_1', 'Difference Variance_2', 'Difference Variance_3',	'Difference Entropy_0','Difference Entropy_1', 'Difference Entropy_2', 'Difference Entropy_3', 'Information Measure of Correlation 1_0','Information Measure of Correlation 1_1', 'Information Measure of Correlation 1_2', 'Information Measure of Correlation 1_3','Information Measure of Correlation 2_0','Information Measure of Correlation 2_1', 'Information Measure of Correlation 2_2', 'Information Measure of Correlation 2_3'], session_id = 1066, fix_imbalance=False, index=False, pca_components=False)



In [ ]:
best_model = compare_models(sort='AUC')


In [ ]:
from pycaret.classification import create_model

# Train the best model
best_model_trained = create_model(best_model)

In [ ]:
df_holdout = pd.read_csv(r"/content/drive/MyDrive/Breast_Segmentation_features/augmentation_approach/augmentation work/cc and mlo segments/cc&mlo_segments_holdout_10.csv")
# df_test.sort_values(by='Date_Time')
df_holdout = df_holdout.drop(df_holdout.loc[:, ['volume', 'case', 'image_name', 'side','view','segment']], axis=1)
display(df_holdout)

In [ ]:
# plot confusion matrix
pycr.plot_model(best_model, plot = 'confusion_matrix')

In [ ]:
# plot AUC
pycr.plot_model(best_model, plot = 'auc')

In [ ]:
# predict on test set
holdout_pred = pycr.predict_model(best_model)

In [ ]:
new_data=df_holdout
#new_data = df_holdout.drop(df_holdout.loc[:, ['volume', 'case', 'image_name', 'side','view','segment','diagnosis']], axis=1)
# new_data = df_new.copy()
# new_data.drop('diagnosis', axis=1, inplace=True)
new_data.head()

In [ ]:
new_predictions = pycr.predict_model(best_model, data = new_data)
new_predictions.head()

In [ ]:
df_test_result = pd.merge(df_holdout, new_predictions[["prediction_label"]], right_index=True, left_index=True)
display(df_test_result)

### Ensemble Models

In [ ]:
model_1 = pycr.create_model('xgboost')
model_1 = pycr.finalize_model(model_1)
model_1_predictions = pycr.predict_model(model_1, data=new_data)

In [ ]:
model_2 = pycr.create_model('rf')
model_2 = pycr.finalize_model(model_2)
model_2_predictions = pycr.predict_model(model_2, data=new_data)

In [ ]:
model_3 = pycr.create_model('et')
model_3 = pycr.finalize_model(model_3)
model_3_predictions = pycr.predict_model(model_3, data=new_data)

In [ ]:
df_temp = pd.merge(model_1_predictions[["prediction_label"]], model_2_predictions[["prediction_label"]], right_index=True, left_index=True)

In [ ]:
df_ensemble = pd.merge(model_3_predictions, df_temp, right_index=True, left_index=True)

In [ ]:
df_ensemble["prediction_final"] = 0
for i, row in df_ensemble.iterrows():
    # check if any 2 out of 3 result columns are equal to 1
    if (row['prediction_label'] + row['prediction_label_x'] + row['prediction_label_y']) >= 2:
        # if yes, set the value of 'new_col' for that row to 1
        df_ensemble.loc[i, 'prediction_final'] = 1

display(df_ensemble)

In [ ]:
df_test_result_ensemble = pd.merge(df_holdout, df_ensemble[["prediction_final"]], right_index=True, left_index=True)
# display(df_test_result_ensemble)
df_test_result_ensemble.to_csv('/content/results.csv', index=True)

In [ ]:
df_ensemble.to_csv('/content/ensemble_test.csv', index=True)

In [ ]:
import pandas as pd
from sklearn.metrics import confusion_matrix, roc_auc_score
from sklearn.metrics import classification_report
from sklearn.metrics import classification_report, accuracy_score

# Load the CSV file
df = pd.read_csv('/content/ensemble_test.csv')
# Extract the true labels and predicted labels
true_labels = df['diagnosis']
predicted_labels = df['prediction_final']

# Create the confusion matrix
cm = confusion_matrix(true_labels, predicted_labels)

# Calculate the AUC
auc = roc_auc_score(true_labels, predicted_labels)
classification_rep = classification_report(true_labels, predicted_labels)

# Assuming you have the predicted labels in 'predicted_labels' and true labels in 'true_labels'

classification_rep = classification_report(true_labels, predicted_labels)
accuracy = accuracy_score(true_labels, predicted_labels)

print("Classification Report:")
print(classification_rep)
print("Accuracy:", accuracy)
# Print the confusion matrix and AUC
print("Confusion Matrix:")
print(cm)
print("AUC:", auc)
print("Classification Report:")
print(classification_rep)